In [ ]:
#data:
#https://quera.org/contest/assignments/32898/problems/109977

In [ ]:
cd drive/

/content/drive


In [ ]:
!unzip '/content/drive/MyDrive/trainset.zip' -d '/content/drive/MyDrive/trainset/'

In [ ]:
from pathlib import Path

# all the current subdirectory
# p = Path('.')
# print([x for x in p.iterdir() if x.is_dir()])
 
data_dir =  Path("drive/MyDrive")
train_dir = data_dir / "trainset" / "trainset"

training_image_paths = list(train_dir.rglob("*.[jJ][pP][gG]")) + list(train_dir.rglob("*.[jJ][pP][eE][gG]"))

print(f'There are {len(training_image_paths)} images in the training set')

There are 9990 images in the training set


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import os, os.path

directory =  '/content/drive/MyDrive/trainset/trainset/'
# images = []
# labels = []
data_len = 9990
images = np.zeros((data_len,224, 224,3), dtype='uint8')
labels = np.zeros((data_len,), dtype='uint8')
names = {}
count = 0
indx = 0
unique = []

shuffled_indices = random.sample(range(0, 9990), 1000)

for i in range(1,11,1):
    folder = directory+str(i)
    for filename in os.listdir(folder):
        print(os.path.join(folder,filename))
        img = cv2.imread(os.path.join(folder,filename))
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        if img is not None and count in shuffled_indices:
            img = cv2.resize(img, (224, 224),interpolation = cv2.INTER_NEAREST)
#             data = np.array(img)
#             print(np.shape(data))
#             flattened = data.flatten()
            
            images[indx] = (img/.256)
            labels[indx] = i
            names[indx] = (filename)
            indx = indx + 1
        count = count + 1 
        

In [ ]:
import sklearn.datasets

dataset = sklearn.utils.Bunch(name = (names) , data=(images), target=(labels))

In [ ]:
X,y = dataset['data'],dataset['target']

from tensorflow.keras.utils import to_categorical

y = to_categorical(y)

print(X.shape)
print(y.shape)

(9990, 224, 224, 3)
(9990, 11)


In [ ]:
def split_train_valid_test_indices(data_len,test_ratio):
    shuffled_indices = np.random.permutation(data_len)
    test_set_size = int(data_len*test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    valid_indices = shuffled_indices[test_set_size:test_set_size*2]
    train_indices = shuffled_indices[test_set_size*2:]
    return list(train_indices), list(valid_indices), list(test_indices)
    

In [ ]:
train_indices , valid_indices , test_indices = split_train_valid_test_indices(X.shape[0],0.2)
train_x = X[train_indices] 
valid_x = X[valid_indices] 
test_x = X[test_indices]
train_y = y[train_indices] 
valid_y = y[valid_indices] 
test_y = y[test_indices]


In [ ]:
# imports for the model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50

# imports for the dataset
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

# Get a pre-built ResNet50 w/o the top layer (classifer) and input shape configured for 128 x 128
base = ResNet50(include_top=False, input_shape=(224, 224, 3), pooling='max')

# Add a new classifier (top) layer for the 10 classes in CIFAR-10
outputs = Dense(11, activation='softmax')(base.output)

# Rebuild the model with the new classifier
resnet = Model(base.input, outputs)
# resnet.summary()

# Create the pre-stem as a Sequential model
model = Sequential()

# Add the ResNet50 model as the remaining layers and rebuild
model.add(resnet)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['acc'])

model.fit(train_x, train_y, epochs=5, batch_size=32, verbose=1, 
                    validation_data=(valid_x, valid_y))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/5
188/188 [==============================] - 74s 362ms/step - loss: 1.3152 - acc: 0.9002 - val_loss: 0.4855 - val_acc: 0.9134
Epoch 2/5
188/188 [==============================] - 68s 359ms/step - loss: 0.3786 - acc: 0.9161 - val_loss: 0.2603 - val_acc: 0.9229
Epoch 3/5
188/188 [==============================] - 63s 334ms/step - loss: 0.2012 - acc: 0.9379 - val_loss: 0.2503 - val_acc: 0.9244
Epoch 4/5
188/188 [==============================] - 63s 333ms/step - loss: 0.1589 - acc: 0.9530 - val_loss: 0.2807 - val_acc: 0.9269
Epoch 5/5
188/188 [==============================] - 67s 359ms/step - loss: 0.0998 - acc: 0.9686 - val_loss: 0.2726 - val_acc: 0.9244


In [ ]:
scores = model.evaluate(test_x, test_y, verbose=1)

63/63 [==============================] - 6s 92ms/step - loss: 0.2501 - acc: 0.9354
